In [31]:
import pandas as pd
import numpy as np
import re

path = "C:/Users/user/Documents/tcc/dados/"
qtd_max_per_features = 10

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
from IPython.display import display_html
from itertools import chain,cycle
import time

def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

def duration_time_parser(string):
    string = str(string)
    string = re.sub('s', '',re.sub(r'h|m', ':', str(string)))
    lista = re.split(':', string, 3)
    if string.lower() == "nan":
        return None         
    elif len(lista) == 1:
        segundos = int(lista[-1])
    elif len(lista) == 2:
        segundos = (int(lista[-2]) * 60) + int(lista[-1])
    elif len(lista) == 3:
        segundos = (int(lista[-3]) * 3600) + (int(lista[-2]) * 60) + int(lista[-1])
    else:
        return None  
    time_format = time.strftime("%H:%M:%S", time.gmtime(segundos))
    return time_format

def duration_time_seconds(string):
    string = str(string)
    string = re.sub('s', '',re.sub(r'h|m', ':', str(string)))
    lista = re.split(':', string, 3)
    if string.lower() == "nan":
        return None         
    elif len(lista) == 1:
        segundos = int(lista[-1])
    elif len(lista) == 2:
        segundos = (int(lista[-2]) * 60) + int(lista[-1])
    elif len(lista) == 3:
        segundos = (int(lista[-3]) * 3600) + (int(lista[-2]) * 60) + int(lista[-1])
    else:
        return None   
    return segundos

In [32]:
df_base = pd.read_csv(f"{path}DF_STREAM_PT_NOME_IDS_wqtdemotes.csv", sep=";")
df_base = df_base.loc[(df_base.total_followers >= 30)] #Publico alvo(canais com pelo menos 60% da meta de afiliado ou mais)
df_base.head()

,broadcaster_id,broadcaster_login,broadcaster_name,total_followers,team_id,qtd_emotes
0,23631109,xruk,xRUK,87,0,0
1,42642296,muuhpro,MuuhPro,229804,0,32
2,62916113,focasp,FocaSP,34720,0,21
3,29361707,zarbaofc,zarbaofc,151,0,0
4,62290505,xtojah,xtojah,8694,0,0


In [33]:
df_tags = pd.read_csv(f"{path}canal_tags.csv", sep=";")
print('qtd registros:', len(df_tags))
print("qtd de tags unicas:", len(df_tags.localization_names.unique())) #há mais de 349 tags nessa base de dados
df_tags = df_tags.drop_duplicates() #tratar registros unicos
print('qtd registros unicos:', len(df_tags))
lista_tags = df_tags[["localization_names", "localization_descriptions"]].drop_duplicates()
print("\n###### df_tags ######")
display(df_tags.head())


df_tags_pivot = pd.crosstab(df_tags['broadcaster_id'],columns=df_tags['localization_names']).astype(int)
print('\n###### df_tags_pivot ######')
display(df_tags_pivot.head(5))


tags_freq = df_tags.localization_names.value_counts(dropna=False).to_frame() \
                    .rename_axis("tags", axis=1) \
                    .rename(columns={'localization_names':'qtd'})
lista_top_50_tags = tags_freq.index.to_list()[:50]
print("\n###### tags_freq ######")                 
display(tags_freq.head(50)) #top 50 tags

is_auto_freq = df_tags.is_auto.value_counts(dropna=False).to_frame() \
                    .rename_axis("sistema_tag", axis=1) \
                    .rename(columns={'is_auto':'qtd'})
print("\n###### is_auto_freq ######")                 
is_auto_freq.head(5) #top 50 tags

qtd registros: 6121
qtd de tags unicas: 314
qtd registros unicos: 6121

###### df_tags ######


,tag_id,is_auto,localization_names,localization_descriptions,broadcaster_id
0,39ee8140-901a-4762-bfca-8260dea1310f,True,Português,Para transmissões que utilizam o idioma português.,42642296
1,39ee8140-901a-4762-bfca-8260dea1310f,True,Português,Para transmissões que utilizam o idioma português.,62916113
2,988c8c1b-b991-4b66-9a6d-b36f60363f6d,False,Brasil,Para transmissões e conteúdos com participação da comunidade brasileira,29361707
3,e1a43486-eee5-4f36-9ee6-76e72af00180,False,Jogada de ranking,Para transmissões focadas em competições em que o objetivo é atingir colocações em placares.,29361707
4,e659959d-392f-44c5-83a5-fb959cdbaccc,False,100%,Para transmissões focadas em completar jogos em 100%,29361707



###### df_tags_pivot ######


localization_names,100%,3DS,AMA,AMM,ASMR auditivo,ASMR visual,Abstinente,Acessibilidade digital,Aconchegante,Acústica,Afeganistão,Afro-americano,Agnóstica,Alemanha,Alemão,Aliado,Andorra,Angola,Animada,Animais,Animação,Anime,Ansiedade,Análise,Análise de rotas,Ao ar livre,Arena,Argentina,Arte com pixel,Arte de fã,Arte digital,Arte tradicional,Arte vetorial,Asiático,Atuação e encenação,Atuação improvisada,Autismo,Autista,Automobilismo,Automotivo,Baixa%,Basquete,Bissexual,Bordado,Brasil,C#,Campanha duradoura,Canadá,Canto,Caridade,Caçando conquistas,Cego,China,Ciclismo,Closed Captions,Clássico,Comentário de partida,Comida de fogão,Comissões de arte,Competitivo,Composição musical,Comédia em standup,Construção,Conteúdo com raids,Controlador,Controle de área,Convenção,Conversa esportiva,Conversas de aquecimento,Conversação,Cooperativo,Cooperação,Coreia,Corrida,Cosplay,Costura,Criação de personagem,Cristão,Cubano,Culinária,Cultura pop,DJ,Dados,Daltonismo,Dance music,Dança contemporânea,Dança tradicional,Decoração de teclado,Deficiência auditiva,Demissexual,Depressão,Desafio das 12 Horas,Desenho,Desenvolvimento de jogos,Desenvolvimento de software,Desenvolvimento na web,Desenvolvimento para desktop,Desenvolvimento para dispositivos móveis,Design,Design gráfico,Diabetes tipo 2,Dificuldade de aprendizagem,Dislexia,Dominica,Draft,Drag,Drones,Drops habilitados,Duplas,EUA,Edição,Educativo,Egito,Ela,Ele,Elu,End Game,Entrevistas,Escrita,Espanha,Espaço seguro,Esportes ao vivo,Esportes radicais,Esports,Esquadrões,Europeu,Exploração,Extensão aprimorada,FGC,Farmar,Festival,Finanças,Foguetes,Fotografia,França,Funk,Furry,Futebol,Fórmula 1,Game Awards,Gay,Gibraltar,Guiné,Gênero feminino,Gênero não binário,Gênero variante,Handebol,Hip hop,História,Homem,House,Hype Train,Idioma: outro,Ilustração,Impressão em 3D,Inglaterra,Inglês,Inscrição de letras,Instrumental,Interativo,Irlanda,Itália,Japonês,Japão,JavaScript,Jazz,Jogabilidade em acesso antecipado,Jogada casual,Jogada com desafio,Jogada de ranking,Jogada pacifista,Jogada randomizada,Jogada sem glitch,Jogadas meméticas,Jogador X ambiente,Jogadores iniciantes,Jogando com espectadores,Jogo de festa,Jogo de guerra,Jogo de tabuleiro clássico,Jogo modificado,Jogo por dinheiro,K-Pop,Kuwait,LGBTQIA+,Latino,Latinos,Leitura em voz alta,Lesão da medula espinhal,Let's Play,Liga,Linux,Luta livre,Luxemburgo,Lésbica,Mapa: Custom,Mapa: Dust II,Mapa: Inferno,Mapa: Mirage,Mapa: Nuke,Mapa: Overpass,Mapa: War Games,Maratona,Metal,Mindfulness,Miniaturas,Modelagem em 3D,Modo difícil,Modo fácil,Modo: Aventura,Modo: Battle Royale,Modo: Campanha,Modo: Criativo,Modo: Salve o Mundo,Modo: Sobrevivência,Modo: Zumbis,Montagem de computadores,Morte permanente,Mulher,Multijogador,Música alternativa,Música eletrônica,Músicos,Nanismo,New Game+,Nigéria,Notícias de jogos,Notícias recentes,Oficina de games,PC,Pais/guardiões,Palpites permitidos,Pansexual,Papo musical,Para toda a família,Personagens de boa índole,Personagens de nível épico,Personagens malignos,Personagens neutros,Pintura,Planejamento de campanha,PlayStation,Portugal,Português,Preto,Primeira jogada,Produção musical,Programação,Prêmio com limite,PvP,Pôquer No Limit,Quadrinhos,Qualquer%,R&B,Rap,Realidade mista,Realidade virtual,Reggae,Reino Unido,Relaxada,Repetir,Retrô,Rock,Russo,Rádio,Rússia,Saúde mental,Sem modificações,Sem palpites,Sem spoilers,Servidor EU,Show competitivo,Skate,Social Eating,Solos,Soul,Speedrun,Spoilers,Squad Stream,Subindo de nível,Suíça,Síndrome de Tourette,TDA,TDAH,Techno,Teclado,Tecnologia assistiva,Tentativas de recorde pessoal,Texas Hold'em,Todos os chefões,Torneio,Torneio de múltiplas mesas,Trabalhos com voz,Trance,Transgênero,Transtorno bipolar,Treinamento,Treino de força,Truques de mágica,Tutorial,Unity,Unreal Engine,Variações de pôquer,Vlogs ao vivo,Vtuber,World music,Xbox,Xbox 360,Xbox One,Xbox Series S/X,Zerg,gamescom,Árabe,Único jogador
broadcaster_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


###### tags_freq ######


tags,qtd
Português,2969
Brasil,374
Portugal,159
Competitivo,118
100%,116
AMA,102
Multijogador,102
LGBTQIA+,95
Jogada casual,89
Drops habilitados,80



###### is_auto_freq ######


sistema_tag,qtd
True,3141
False,2980


In [34]:
df_clips = pd.read_csv(f"{path}clips_2022-05-20_a_2022-08-20.csv", sep=";", encoding='iso-8859-1') #Os clips estão sem categoria (usar só max e média?)
df_clips = df_clips.sort_values(by=['view_count'], ascending=False)
df_clips.head(5)

,id,url,embed_url,broadcaster_id,broadcaster_name,creator_id,creator_name,video_id,game_id,language,title,view_count,created_at,thumbnail_url,duration,pagination
3012,StylishSullenHabaneroArgieB8-5uuntQg5LCQ38J6n,https://clips.twitch.tv/StylishSullenHabaneroArgieB8-5uuntQg5LCQ38J6n,https://clips.twitch.tv/embed?clip=StylishSullenHabaneroArgieB8-5uuntQg5LCQ38J6n,75947497,s2alinefaria,254193053,kschorro_do_mangue,NaN,116747788.0,pt-br,entreguei o baiano,91909,2022-08-09T22:45:39Z,https://clips-media-assets2.twitch.tv/qba2Lebrp7UIOSwkN_Cepw/40046882777-offset-1574-preview-480x272.jpg,30.0,NaN
3013,AnnoyingJollyIcecreamLeeroyJenkins-LsPg5AZlaEEDLZ2a,https://clips.twitch.tv/AnnoyingJollyIcecreamLeeroyJenkins-LsPg5AZlaEEDLZ2a,https://clips.twitch.tv/embed?clip=AnnoyingJollyIcecreamLeeroyJenkins-LsPg5AZlaEEDLZ2a,75947497,s2alinefaria,174990494,Daviizaoo,NaN,116747788.0,pt-br,veguinha,62932,2022-08-04T23:35:30Z,https://clips-media-assets2.twitch.tv/O3-N-S7D4OQGZWhBAy2dmg/40022187881-offset-894-preview-480x272.jpg,30.0,NaN
3014,BrainyAdorableWaffleTTours-TuE8GpNC4VWJTto4,https://clips.twitch.tv/BrainyAdorableWaffleTTours-TuE8GpNC4VWJTto4,https://clips.twitch.tv/embed?clip=BrainyAdorableWaffleTTours-TuE8GpNC4VWJTto4,75947497,s2alinefaria,799414038,bernardin444344,NaN,116747788.0,pt-br,ta krai,51280,2022-08-09T22:45:33Z,https://clips-media-assets2.twitch.tv/UD0wg07hPBqTQuFViU7aOA/AT-cm%7CUD0wg07hPBqTQuFViU7aOA-preview-480x272.jpg,59.9,NaN
3015,TransparentBraveEggBrokeBack-yx8jW_ONKIvTQZqL,https://clips.twitch.tv/TransparentBraveEggBrokeBack-yx8jW_ONKIvTQZqL,https://clips.twitch.tv/embed?clip=TransparentBraveEggBrokeBack-yx8jW_ONKIvTQZqL,75947497,s2alinefaria,109277671,bronnks7,NaN,116747788.0,pt-br,kkkk,33292,2022-08-09T23:10:04Z,https://clips-media-assets2.twitch.tv/Bzdf-csfI1vzjSiL00Gaiw/40046882777-offset-3040-preview-480x272.jpg,30.0,NaN
52182,ShinyPolishedLatteSeemsGood-YtYjXnOoZogrx81Y,https://clips.twitch.tv/ShinyPolishedLatteSeemsGood-YtYjXnOoZogrx81Y,https://clips.twitch.tv/embed?clip=ShinyPolishedLatteSeemsGood-YtYjXnOoZogrx81Y,122675528,Hirowdots,53235445,loogw,1.513269e+09,509658.0,pt-br,parabens,32897,2022-06-25T02:52:56Z,https://clips-media-assets2.twitch.tv/AT-cm%7Cnfk4emLWI15zqGaIK8fNWA-preview-480x272.jpg,42.9,6424.0


# Tratamento categorias/capitulos de Video

In [35]:
df_videos = pd.read_csv(f"{path}videos_with_chapters_novo.csv", sep=";", encoding='iso-8859-1') #pode ser que alguns registros tenham um titulo que prejudiquem o dataframe df_videos['view_count'] == 'pt' (a base de treinamento foi corrida manualmente, mas pode poder ocorrer em "produção")
print('qtd registros df_videos:', df_videos.shape)
display(df_videos.head(5))
df_videos['duration_time'] = df_videos.duration.apply(duration_time_parser)
df_videos['duration_seconds'] = df_videos.duration.apply(duration_time_seconds)


qtd registros df_videos: (47432, 20)


,id,stream_id,user_id,user_login,user_name,title,description,created_at,published_at,url,thumbnail_url,viewable,view_count,language,type,duration,muted_segments,game_id,game_name,game_duration
0,1566966657,1,42642296,muuhpro,MuuhPro,Hazel Sky - Parte 1/InÃ­cio,NaN,2022-08-20 00:45:25+00:00,2022-08-20T00:45:25Z,https://www.twitch.tv/videos/1566966657,https://static-cdn.jtvnw.net/cf_vods/d2nvs31859zcd8/8f9e922425c77771c284_muuhpro_33396531370_5866245686/thumb/custom-0f22b465-a861-4969-96c6-7c3fc906d1c2-%{width}x%{height}.jpeg,public,67.0,pt,VideoType.HIGHLIGHT,24m47s,NaN,516587.0,Hazel Sky,NaN
1,1566715632,1,62916113,focasp,FocaSP,Destaque: !potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-19 19:56:50+00:00,2022-08-19T19:56:50Z,https://www.twitch.tv/videos/1566715632,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/7bdce5238ad5e779a2ad_focasp_93063639266_2432970080//thumb/thumb1566715632-%{width}x%{height}.jpg,public,2.0,pt,VideoType.HIGHLIGHT,4h11m20s,NaN,32982.0,Grand Theft Auto V,NaN
2,1566107414,40091482665,62916113,focasp,FocaSP,!potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-19 02:25:36+00:00,2022-08-19T02:25:36Z,https://www.twitch.tv/videos/1566107414,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/3aee2950b0bd938bd7a9_focasp_40091482665_1660875932//thumb/thumb0-%{width}x%{height}.jpg,public,3090.0,pt,VideoType.ARCHIVE,4h11m20s,NaN,32982.0,Grand Theft Auto V,NaN
3,1560046282,1,62916113,focasp,FocaSP,Destaque: !potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-12 19:42:09+00:00,2022-08-12T19:42:09Z,https://www.twitch.tv/videos/1560046282,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/96293e4eff7e9414bfb3_focasp_66270880770_9823912542//thumb/thumb1560046282-%{width}x%{height}.jpg,public,3.0,pt,VideoType.HIGHLIGHT,3h18m40s,NaN,32982.0,Grand Theft Auto V,NaN
4,1559385187,40057427737,62916113,focasp,FocaSP,!potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-12 01:19:58+00:00,2022-08-12T01:19:58Z,https://www.twitch.tv/videos/1559385187,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/9a87c18cee3a1add5dd0_focasp_40057427737_1660267194//thumb/thumb0-%{width}x%{height}.jpg,public,2366.0,pt,VideoType.ARCHIVE,3h18m40s,NaN,32982.0,Grand Theft Auto V,NaN


In [36]:
metrics_videos = df_videos[['user_id', 'view_count', 'duration', 'duration_time', 'duration_seconds']]
metrics_videos = metrics_videos.sort_values(by=['duration'])

metrics_videos_duration = metrics_videos.groupby('user_id').agg({'duration_seconds': ['mean', 'min', 'max']})
metrics_videos_duration.columns = ['_'.join(col) for col in metrics_videos_duration.columns.values]
metrics_videos_duration = metrics_videos_duration.reset_index(level=0).rename(columns={"user_id": "broadcaster_id"})

metrics_videos_view = metrics_videos.groupby('user_id').agg({'view_count': ['mean', 'min', 'max', 'count']})
metrics_videos_view.columns = ['_'.join(col) for col in metrics_videos_view.columns.values]
metrics_videos_view = metrics_videos_view.reset_index(level=0).rename(columns={"user_id": "broadcaster_id"})

qtd_exibicao = 10
display_side_by_side(metrics_videos_duration[:qtd_exibicao],metrics_videos_view[:qtd_exibicao], titles=['Duration Metrics','View Metrics'])
print(f'\nShowing {qtd_exibicao} rows')

,broadcaster_id,duration_seconds_mean,duration_seconds_min,duration_seconds_max
0,646160,1240.000000,1240.0,1240.0
1,1220595,9770.689655,7642.0,15238.0
2,2040044,4993.000000,3102.0,7851.0
3,2045016,7741.000000,7256.0,8494.0
4,2368252,7453.000000,154.0,18732.0
5,2605367,656.000000,656.0,656.0
6,3083045,23.000000,23.0,23.0
7,3393831,3661.000000,607.0,10301.0
8,3458602,8924.000000,8924.0,8924.0
9,3533438,46516.891892,12984.0,153258.0



Showing 10 rows


In [37]:
print("qtd de games unicos:", len(df_videos.game_name.unique()))
df_videos = df_videos.drop_duplicates() #tratar registros unicos
print('qtd registros unicos:', len(df_videos))
lista_games = df_videos[["game_id", "game_name"]].drop_duplicates()
print("\n###### df_videos ######")
display(df_videos.head())


df_games_pivot = pd.crosstab(df_videos['user_id'],columns=df_videos['game_name']).astype(int)
print('\n###### df_games_pivot ######')
display(df_games_pivot.head(5))


df_videos_freq = df_videos.game_name.value_counts(dropna=False).to_frame() \
                    .rename_axis("games", axis=1) \
                    .rename(columns={'game_name':'qtd'})
print("\n###### tags_freq ######")                 
df_videos_freq.head(50) #top 50 games mais jogados no periodo de extraçao dos videos

qtd de games unicos: 1260
qtd registros unicos: 47432

###### df_videos ######


,id,stream_id,user_id,user_login,user_name,title,description,created_at,published_at,url,thumbnail_url,viewable,view_count,language,type,duration,muted_segments,game_id,game_name,game_duration,duration_time,duration_seconds
0,1566966657,1,42642296,muuhpro,MuuhPro,Hazel Sky - Parte 1/InÃ­cio,NaN,2022-08-20 00:45:25+00:00,2022-08-20T00:45:25Z,https://www.twitch.tv/videos/1566966657,https://static-cdn.jtvnw.net/cf_vods/d2nvs31859zcd8/8f9e922425c77771c284_muuhpro_33396531370_5866245686/thumb/custom-0f22b465-a861-4969-96c6-7c3fc906d1c2-%{width}x%{height}.jpeg,public,67.0,pt,VideoType.HIGHLIGHT,24m47s,NaN,516587.0,Hazel Sky,NaN,00:24:47,1487.0
1,1566715632,1,62916113,focasp,FocaSP,Destaque: !potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-19 19:56:50+00:00,2022-08-19T19:56:50Z,https://www.twitch.tv/videos/1566715632,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/7bdce5238ad5e779a2ad_focasp_93063639266_2432970080//thumb/thumb1566715632-%{width}x%{height}.jpg,public,2.0,pt,VideoType.HIGHLIGHT,4h11m20s,NaN,32982.0,Grand Theft Auto V,NaN,04:11:20,15080.0
2,1566107414,40091482665,62916113,focasp,FocaSP,!potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-19 02:25:36+00:00,2022-08-19T02:25:36Z,https://www.twitch.tv/videos/1566107414,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/3aee2950b0bd938bd7a9_focasp_40091482665_1660875932//thumb/thumb0-%{width}x%{height}.jpg,public,3090.0,pt,VideoType.ARCHIVE,4h11m20s,NaN,32982.0,Grand Theft Auto V,NaN,04:11:20,15080.0
3,1560046282,1,62916113,focasp,FocaSP,Destaque: !potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-12 19:42:09+00:00,2022-08-12T19:42:09Z,https://www.twitch.tv/videos/1560046282,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/96293e4eff7e9414bfb3_focasp_66270880770_9823912542//thumb/thumb1560046282-%{width}x%{height}.jpg,public,3.0,pt,VideoType.HIGHLIGHT,3h18m40s,NaN,32982.0,Grand Theft Auto V,NaN,03:18:40,11920.0
4,1559385187,40057427737,62916113,focasp,FocaSP,!potatosrp | Seu Adalberto em serviÃ§o <3,NaN,2022-08-12 01:19:58+00:00,2022-08-12T01:19:58Z,https://www.twitch.tv/videos/1559385187,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/9a87c18cee3a1add5dd0_focasp_40057427737_1660267194//thumb/thumb0-%{width}x%{height}.jpg,public,2366.0,pt,VideoType.ARCHIVE,3h18m40s,NaN,32982.0,Grand Theft Auto V,NaN,03:18:40,11920.0



###### df_games_pivot ######


game_name,100% Orange Juice,60 Parsecs!,7 Days to Die,80 Days,A Girls Fabric Face,A Hat in Time,A Plague Tale: Innocence,A Way Out,A pet shop after dark,ANVIL,ASMR,Ace Combat 5: The Unsung War,Aero Tales Online: The World,Aeterna Noctis,After the Fall,Age of Empires II,Age of Empires III,Age of Empires IV,Age of Empires Online,Age of Mythology,Aim Lab,Aion,Aka,Alaloth: Champions of the Four Kingdoms,Alan Wake,Albion Online,Alchemy Stars,Alien: Isolation,Aliens: Fireteam Elite,Alwa's Awakening,American McGee's Alice,American Truck Simulator,Amnesia: Rebirth,Among Us,Ancestors: The Humankind Odyssey,Ancient Gods,Andro Dunos II,Anger Foot,Animal Crossing: New Horizons,Anvil Saga,Apex Legends,Apex Legends Mobile,Apocalypse,Arcade Paradise,Arcadegeddon,ArcheAge,Archvale,Arena of Valor,Arizona Sunshine,Ark: Survival Evolved,Arknights,Arma 3,As Dusk Falls,Assassin's Creed III,Assassin's Creed III Remastered,Assassin's Creed Odyssey,Assassin's Creed Valhalla,Assetto Corsa,Assetto Corsa Competizione,Astroneer,Axie Infinity,Axiom Verge,Azure Striker GUNVOLT 2,Babylon's Fall,Back 4 Blood,Bad Dudes,Baldur's Gate 3,Balloon Flight,Bartender Hustle,Batman: Arkham Knight,Batman: The Telltale Series,Battalion 1944,Battle Brothers,BattleBit,Battlefield 1,Battlefield 2042,Battlefield 3,Battlefield 4,Battlefield V,Bear & Breakfast,Beat Saber,Beats Of Fury,Beneath Oresa,Beneath the Mountain,Beyond: Two Souls,Big Earth,Big Time,BioShock Remastered,Biomutant,Bishoujo Senshi Sailor Moon: Another Story,Black Desert,Black Desert Mobile,Black Mesa,Black Squad,Bladed Fury,Blair Witch,Blasphemous,BlazBlue: Central Fiction,Bloodborne,Bloodstained: Ritual of the Night,Bloodstone Online: The Ancient Curse,Bloons TD 6,Blue Archive,Board Games,BombParty,Borderlands 2,Borderlands 3,Borderlands: The Pre-Sequel,Brasfoot 2022,Brave Fencer Musashi,Brawl Stars,Brawlhalla,Breakers Revenge,Bright Memory: Infinite,Broforce,Brothers: A Tale of Two Sons,Bugsnax,Bully,Bum Simulator,Business Tour,CABAL Online,CHANGE: A Homeless Survival Experience,Cafe Owner Simulator,Caliber,Call Of Duty: Modern Warfare,Call of Duty: Black Ops,Call of Duty: Black Ops Cold War,Call of Duty: Black Ops II,Call of Duty: Black Ops III,Call of Duty: Black Ops Zombies,Call of Duty: Mobile,Call of Duty: Modern Warfare II,Call of Duty: Vanguard,Call of Duty: Warzone,Call to Arms: Gates of Hell,Capcom Arcade 2nd Stadium,Capcom Arcade Stadium,Captain Toad: Treasure Tracker,Captain Velvet Meteor: The Jump+ Dimensions,Car Detailing Simulator,CarX Drift Racing Online,Card Games,Cards of the Dead,Cartel Tycoon,Castle Crush: Epic Strategy Game,Castle Pals,Castlevania,Castlevania Chronicles,Castlevania III: Dracula's Curse,Castlevania: Bloodlines,Castlevania: Circle of the Moon,Castlevania: Harmony of Despair,Castlevania: Lords of Shadow,Castlevania: Symphony of the Night,Catizens,Celeste,Century: Age of Ashes,Chernobylite,Chess,Chess.com,ChessLocke,Children of Morta,Chimeraland,Chivalry 2,Chronicon,Chrono Trigger,Clash Royale,Clash of Clans,Clone Hero,Clustertruck,Code Vein,Coin Operated Carnage,Company of Heroes 2,Conan Exiles,Conquer Online,Conquer Online 2.0,Conqueror's Blade,Contexto,Contractors,Control,"Cook, Serve, Delicious! 3?!",Cookie Clicker,Cooking Simulator,Core Keeper,Coromon,Cosmic Star Heroine,Cosmodread,Counter-Strike,Counter-Strike: Global Offensive,Counter-Strike: Source,Crab Game,Crash Bandicoot,Crash Bandicoot 4: It's About Time,Crash Bandicoot: Warped,Crash Team Racing,Crash Team Racing Nitro-Fueled,Crashlands,Creativerse,Cris Tales,Crossfire,Crossfire X,Crossout,Crowz,Crusader Kings III,Cry of Fear,Crypt of the NecroDancer,Crypto,Crysis Remastered,Cult of the Lamb,Cultic,Cultist Simulator,Cuphead,Cursed to Golf,CyberCorp,Cyberpunk 2077,DCS World,DDTank,DEATHLOOP,DEVOUR,DIRT 5,DJMAX Respect,DNF Duel,DOOM,DOOM Eternal,Dakar 18,Dante's Inferno,Dark Souls,Dark Souls II,Dark Souls III,Dark Souls: Remastered,Darkest Dungeon,Darksiders Genesis,Darksiders II: Deathinitive Edition,Darksiders


###### tags_freq ######


games,qtd
Grand Theft Auto V,3746
VALORANT,3588
League of Legends,3512
Counter-Strike: Global Offensive,2297
Tibia,1709
Fortnite,1466
FIFA 22,1259
Call of Duty: Warzone,1059
Dead by Daylight,928
PUBG: BATTLEGROUNDS,920


In [38]:
test = df_videos[['user_id','game_id', 'created_at', 'published_at']]
test.loc[test.user_id == 53052202].sort_values('created_at').head(20)

,user_id,game_id,created_at,published_at
17228,53052202,32399.0,2022-06-28 16:53:14+00:00,2022-06-28T16:53:14Z
17227,53052202,32399.0,2022-06-29 22:17:21+00:00,2022-06-29T22:17:21Z
17226,53052202,32399.0,2022-06-30 14:39:35+00:00,2022-06-30T14:39:35Z
17225,53052202,32399.0,2022-06-30 23:22:42+00:00,2022-06-30T23:22:42Z
17224,53052202,32399.0,2022-07-02 23:20:26+00:00,2022-07-02T23:20:26Z
17223,53052202,32399.0,2022-07-27 00:11:37+00:00,2022-07-27T00:11:37Z
17222,53052202,32399.0,2022-07-28 00:36:10+00:00,2022-07-28T00:36:10Z
17221,53052202,32399.0,2022-07-29 00:16:19+00:00,2022-07-29T00:16:19Z
17220,53052202,32399.0,2022-08-01 00:50:55+00:00,2022-08-01T00:50:55Z
17219,53052202,32399.0,2022-08-01 01:19:12+00:00,2022-08-01T01:19:12Z


In [39]:
rank_jogos_recentes = df_videos[['user_id','game_id', 'created_at']].copy()
#coluna order criada para casos das lives com mais de um capitulo e tem a mesma data de criação por pertecerem a mesma transmissão
#pode ser considerada "ordem" de registros pois a extração do web scrapping foi feita em ordem dos elementos de capitulo
rank_jogos_recentes['default_rank'] = rank_jogos_recentes.index + 1
rank_jogos_recentes["temp"] = rank_jogos_recentes[["created_at","default_rank"]].apply(tuple,axis=1)\
                                                 .rank(method='dense',ascending=True).astype(int)
rank_jogos_recentes['rank_jogos_recentes'] = rank_jogos_recentes.groupby(['user_id'])['temp'].rank("max", ascending=False).astype(int)

rank_jogos_recentes_pivot = pd.pivot_table(rank_jogos_recentes.loc[rank_jogos_recentes.rank_jogos_recentes <= qtd_max_per_features]
                                        ,values='game_id' \
                                        ,index='user_id' \
                                        ,columns='rank_jogos_recentes' \
                                        ,aggfunc=np.sum \
                                        ,fill_value=0).astype(int)

rank_jogos_recentes_pivot = rank_jogos_recentes_pivot.rename_axis("index", axis="columns").reset_index()
rank_jogos_recentes_pivot.loc[rank_jogos_recentes_pivot.user_id == 53052202]
rank_jogos_recentes_pivot = rank_jogos_recentes_pivot.add_prefix('rank_jogos_recentes_').rename(columns={"rank_jogos_recentes_user_id":"user_id"})

rank_jogos_recentes_pivot.loc[rank_jogos_recentes_pivot.user_id == 53052202].head(20)

index,user_id,rank_jogos_recentes_1,rank_jogos_recentes_2,rank_jogos_recentes_3,rank_jogos_recentes_4,rank_jogos_recentes_5,rank_jogos_recentes_6,rank_jogos_recentes_7,rank_jogos_recentes_8,rank_jogos_recentes_9,rank_jogos_recentes_10
1094,53052202,32399,32399,32399,32399,32399,32399,32399,32399,32399,32399


In [40]:
rank_jogos_recentes

,user_id,game_id,created_at,default_rank,temp,rank_jogos_recentes
0,42642296,516587.0,2022-08-20 00:45:25+00:00,1,45737,1
1,62916113,32982.0,2022-08-19 19:56:50+00:00,2,45158,1
2,62916113,32982.0,2022-08-19 02:25:36+00:00,3,44233,2
3,62916113,32982.0,2022-08-12 19:42:09+00:00,4,36001,3
4,62916113,32982.0,2022-08-12 01:19:58+00:00,5,35259,4
...,...,...,...,...,...,...
47427,231313969,494925.0,2022-07-14 13:33:58+00:00,47428,11057,10
47428,231313969,32399.0,2022-06-10 11:22:05+00:00,47429,699,11
47429,231313969,488479.0,2022-06-09 23:08:42+00:00,47430,689,12
47430,222575867,512953.0,2022-08-20 17:43:25+00:00,47431,46843,1


In [41]:
metrics_game_view = df_videos.rename(columns={'duration_seconds':'game_duration_seconds'}) \
                            .groupby(['user_id','game_id','game_name',]) \
                            .agg({'game_duration_seconds': ['min', 'max', 'mean','sum', 'count']})
metrics_game_view.columns = ['_'.join(col) for col in metrics_game_view.columns.values]
# metrics_game_view = metrics_game_view.reset_index(level=0).rename(columns={"user_id": "broadcaster_id"})
metrics_game_view = metrics_game_view.reset_index()
metrics_game_view['game_id'] = metrics_game_view['game_id'].astype(int)

In [42]:
metrics_game_view

,user_id,game_id,game_name,game_duration_seconds_min,game_duration_seconds_max,game_duration_seconds_mean,game_duration_seconds_sum,game_duration_seconds_count
0,646160,514111,Hatsune Miku: Project DIVA Mega Mix,1240.0,1240.0,1240.000000,1240.0,1
1,1220595,66402,Battlefield 4,8286.0,9748.0,9017.000000,18034.0,2
2,1220595,488500,Battlefield 1,9118.0,13394.0,11256.000000,22512.0,2
3,1220595,490379,Tom Clancy's Ghost Recon: Wildlands,9050.0,9050.0,9050.000000,9050.0,1
4,1220595,491705,Insurgency: Sandstorm,7642.0,7642.0,7642.000000,7642.0,1
...,...,...,...,...,...,...,...,...
7222,812050263,491487,Dead by Daylight,44.0,44.0,44.000000,44.0,1
7223,812050263,512953,Elden Ring,5448.0,35248.0,20348.000000,40696.0,2
7224,816551851,502732,Garena Free Fire,8638.0,10612.0,9468.666667,28406.0,3
7225,816551851,516575,VALORANT,594.0,11824.0,4961.333333,14884.0,3


In [43]:
metrics_game_view = metrics_game_view.reset_index()
metrics_game_view

,index,user_id,game_id,game_name,game_duration_seconds_min,game_duration_seconds_max,game_duration_seconds_mean,game_duration_seconds_sum,game_duration_seconds_count
0,0,646160,514111,Hatsune Miku: Project DIVA Mega Mix,1240.0,1240.0,1240.000000,1240.0,1
1,1,1220595,66402,Battlefield 4,8286.0,9748.0,9017.000000,18034.0,2
2,2,1220595,488500,Battlefield 1,9118.0,13394.0,11256.000000,22512.0,2
3,3,1220595,490379,Tom Clancy's Ghost Recon: Wildlands,9050.0,9050.0,9050.000000,9050.0,1
4,4,1220595,491705,Insurgency: Sandstorm,7642.0,7642.0,7642.000000,7642.0,1
...,...,...,...,...,...,...,...,...,...
7222,7222,812050263,491487,Dead by Daylight,44.0,44.0,44.000000,44.0,1
7223,7223,812050263,512953,Elden Ring,5448.0,35248.0,20348.000000,40696.0,2
7224,7224,816551851,502732,Garena Free Fire,8638.0,10612.0,9468.666667,28406.0,3
7225,7225,816551851,516575,VALORANT,594.0,11824.0,4961.333333,14884.0,3


In [44]:
# # metrics_game_view.loc[metrics_game_view['game_duration_seconds_count'].idxmax()]
# # metrics_game_view['game_max_count'] = metrics_game_view.groupby(['user_id'])['game_duration_seconds_count'].transform(max)\
# # metrics_game_view.groupby(['user_id'])['game_duration_seconds_count'].transform(max) == metrics_game_view['game_duration_seconds_count']
# game_max_count = metrics_game_view[metrics_game_view.groupby(['user_id'])['game_duration_seconds_count'].transform(max) == metrics_game_view['game_duration_seconds_count']][['user_id', 'game_id', 'game_name', 'game_duration_seconds_count']]
# game_max_sum = metrics_game_view[metrics_game_view.groupby(['user_id'])['game_duration_seconds_sum'].transform(max) == metrics_game_view['game_duration_seconds_sum']][['user_id', 'game_id', 'game_name', 'game_duration_seconds_sum']]
# # game_max_sum

#^^^^^^^^^^^^^^^^ esses nao precisam pois tao rankeados



In [45]:
metrics_game_view[f'rank_game_duration_seconds_sum'] = metrics_game_view.groupby(['user_id'])['game_duration_seconds_sum'].rank("max", ascending=False).astype(int)
metrics_game_view[f'rank_game_duration_seconds_count'] = metrics_game_view.groupby(['user_id'])['game_duration_seconds_count'].rank("max", ascending=False).astype(int)
metrics_game_view[f'rank_game_duration_seconds_mean'] = metrics_game_view.groupby(['user_id'])['game_duration_seconds_mean'].rank("max", ascending=False).astype(int)

metrics_game_view[f'rank_game_duration_seconds_min'] = metrics_game_view.groupby(['user_id'])['game_duration_seconds_min'].rank("max", ascending=True).astype(int)
metrics_game_view[f'rank_game_duration_seconds_max'] = metrics_game_view.groupby(['user_id'])['game_duration_seconds_max'].rank("max", ascending=False).astype(int)


In [46]:
metrics_game_view.loc[metrics_game_view.user_id == 53052202].sort_values('game_duration_seconds_sum').head(20)

,index,user_id,game_id,game_name,game_duration_seconds_min,game_duration_seconds_max,game_duration_seconds_mean,game_duration_seconds_sum,game_duration_seconds_count,rank_game_duration_seconds_sum,rank_game_duration_seconds_count,rank_game_duration_seconds_mean,rank_game_duration_seconds_min,rank_game_duration_seconds_max
2480,2480,53052202,32399,Counter-Strike: Global Offensive,1650.0,105608.0,21206.8125,339309.0,16,1,1,1,1,1


In [47]:
menor_trecho_game = metrics_game_view[['user_id', 'game_id', 'game_name', 'rank_game_duration_seconds_min']] \
                        .loc[metrics_game_view['rank_game_duration_seconds_min'] == 1] \
                        .rename(columns={"game_id":"menor_trecho_game"}) \
                        .drop(columns={"game_name", "rank_game_duration_seconds_min"}) \
                        .reset_index() \
                        .drop(columns=['index'])

maior_trecho_game = metrics_game_view[['user_id', 'game_id', 'game_name', 'rank_game_duration_seconds_max']] \
                    .loc[metrics_game_view['rank_game_duration_seconds_max'] == 1] \
                    .rename(columns={"game_id":"maior_trecho_game"}) \
                    .drop(columns={"game_name", "rank_game_duration_seconds_max"}) \
                    .reset_index() \
                    .drop(columns=['index'])

In [48]:
menor_trecho_game.loc[menor_trecho_game.user_id == 53052202]

,user_id,menor_trecho_game
1090,53052202,32399


In [49]:
pivot_base_df = metrics_game_view[['user_id', 'game_id', 'game_name', 'rank_game_duration_seconds_sum', 'rank_game_duration_seconds_count', 'rank_game_duration_seconds_mean']]
pivot_base_df

,user_id,game_id,game_name,rank_game_duration_seconds_sum,rank_game_duration_seconds_count,rank_game_duration_seconds_mean
0,646160,514111,Hatsune Miku: Project DIVA Mega Mix,1,1,1
1,1220595,66402,Battlefield 4,6,6,13
2,1220595,488500,Battlefield 1,4,6,3
3,1220595,490379,Tom Clancy's Ghost Recon: Wildlands,14,17,12
4,1220595,491705,Insurgency: Sandstorm,17,17,17
...,...,...,...,...,...,...
7222,812050263,491487,Dead by Daylight,2,2,2
7223,812050263,512953,Elden Ring,1,1,1
7224,816551851,502732,Garena Free Fire,1,2,1
7225,816551851,516575,VALORANT,2,2,2


In [52]:
game_duration_sum_rank = pd.pivot_table(pivot_base_df.loc[pivot_base_df.rank_game_duration_seconds_sum <= qtd_max_per_features]
                                        ,values='game_id' \
                                        ,index='user_id' \
                                        ,columns='rank_game_duration_seconds_sum' \
                                        ,aggfunc=np.sum \
                                        ,fill_value=0).astype(int)

game_duration_sum_rank = game_duration_sum_rank.rename_axis("index", axis="columns").reset_index()
game_duration_sum_rank = game_duration_sum_rank.add_prefix('game_duration_sum_rank_').rename(columns={"game_duration_sum_rank_user_id":"user_id"})


game_duration_count_rank = pd.pivot_table(pivot_base_df.loc[pivot_base_df.rank_game_duration_seconds_count <= qtd_max_per_features]
                                        ,values='game_id' \
                                        ,index='user_id' \
                                        ,columns='rank_game_duration_seconds_count' \
                                        ,aggfunc=np.sum \
                                        ,fill_value=0).astype(int)

game_duration_count_rank = game_duration_count_rank.rename_axis("index", axis="columns").reset_index()
game_duration_count_rank = game_duration_count_rank.add_prefix('game_duration_count_rank_').rename(columns={"game_duration_count_rank_user_id":"user_id"})


game_duration_mean_rank = pd.pivot_table(pivot_base_df.loc[pivot_base_df.rank_game_duration_seconds_mean <= qtd_max_per_features]
                                        ,values='game_id' \
                                        ,index='user_id' \
                                        ,columns='rank_game_duration_seconds_mean' \
                                        ,aggfunc=np.sum \
                                        ,fill_value=0).astype(int)

game_duration_mean_rank = game_duration_mean_rank.rename_axis("index", axis="columns").reset_index()
game_duration_mean_rank = game_duration_mean_rank.add_prefix('game_duration_mean_rank_').rename(columns={"game_duration_mean_rank_user_id":"user_id"})

qtd_exibicao = 10
display_side_by_side(game_duration_sum_rank[:qtd_exibicao],game_duration_count_rank[:qtd_exibicao],game_duration_mean_rank[:qtd_exibicao], titles=['Top 10 total gameplay','Top 10 frequencia gameplay', 'Top 10 media gameplay'])
print(f'Showing {qtd_exibicao} rows')

index,user_id,game_duration_sum_rank_1,game_duration_sum_rank_2,game_duration_sum_rank_3,game_duration_sum_rank_4,game_duration_sum_rank_5,game_duration_sum_rank_6,game_duration_sum_rank_7,game_duration_sum_rank_8,game_duration_sum_rank_9,game_duration_sum_rank_10
0,646160,514111,0,0,0,0,0,0,0,0,0
1,1220595,497078,514974,504199,488500,509249,66402,508776,512782,517770,1757732267
2,2040044,497057,1393686646,0,0,0,0,0,0,0,0
3,2045016,18122,0,0,0,0,0,0,0,0,0
4,2368252,19619,514374,0,0,0,0,0,0,0,0
5,2605367,370472,0,0,0,0,0,0,0,0,0
6,3083045,18122,0,0,0,0,0,0,0,0,0
7,3393831,1869092879,0,0,0,0,0,0,0,0,0
8,3458602,1061323687,0,0,0,0,0,0,0,0,0
9,3533438,512710,513181,511224,491931,0,0,0,0,0,0


Showing 10 rows


In [53]:
game_duration_sum_rank.loc[game_duration_sum_rank.user_id == 53052202]

index,user_id,game_duration_sum_rank_1,game_duration_sum_rank_2,game_duration_sum_rank_3,game_duration_sum_rank_4,game_duration_sum_rank_5,game_duration_sum_rank_6,game_duration_sum_rank_7,game_duration_sum_rank_8,game_duration_sum_rank_9,game_duration_sum_rank_10
1094,53052202,32399,0,0,0,0,0,0,0,0,0


In [54]:
df_base.rename(columns={"broadcaster_id":"user_id"})

,user_id,broadcaster_login,broadcaster_name,total_followers,team_id,qtd_emotes
0,23631109,xruk,xRUK,87,0,0
1,42642296,muuhpro,MuuhPro,229804,0,32
2,62916113,focasp,FocaSP,34720,0,21
3,29361707,zarbaofc,zarbaofc,151,0,0
4,62290505,xtojah,xtojah,8694,0,0
...,...,...,...,...,...,...
14383,702031641,m3ktrefe,m3ktrefe,50,0,0
14384,38900392,rafaolanches,RafaoLanches,678,0,2
14385,231313969,vgh0st,vgh0st,807,5694,16
14386,222575867,ironvirus19,ironvirus19,494,0,0


In [55]:
print('canais publico alvo', df_base.shape)
print('canais com info de video:', game_duration_sum_rank.shape)
print('canais com tags:', df_tags_pivot.shape)

df_videos_all_metrics = df_base.rename(columns={"broadcaster_id":"user_id"}) \
                                         .merge(df_tags_pivot[lista_top_50_tags].reset_index().rename(columns={"broadcaster_id":"user_id"}), on='user_id', how='left') \
                                         .merge(menor_trecho_game, on='user_id', how='left') \
                                         .merge(maior_trecho_game, on='user_id', how='left') \
                                         .merge(game_duration_sum_rank.reset_index().drop(columns='index'), on='user_id', how='left') \
                                         .merge(game_duration_count_rank.reset_index().drop(columns='index'), on='user_id', how='left') \
                                         .merge(game_duration_mean_rank.reset_index().drop(columns='index'), on='user_id', how='left') \
                                         .merge(rank_jogos_recentes_pivot.reset_index().drop(columns='index'), on='user_id', how='left') \
                                         .rename(columns={"user_id":"broadcaster_id"})

df_videos_all_metrics

canais publico alvo (14388, 6)
canais com info de video: (3397, 11)
canais com tags: (3997, 314)


,broadcaster_id,broadcaster_login,broadcaster_name,total_followers,team_id,qtd_emotes,Português,Brasil,Portugal,Competitivo,100%,AMA,Multijogador,LGBTQIA+,Jogada casual,Drops habilitados,Conversação,PvP,Jogando com espectadores,Anime,Jogada de ranking,Atuação e encenação,PC,Jogadores iniciantes,Primeira jogada,Futebol,Aconchegante,Relaxada,Inglês,Único jogador,Modo: Battle Royale,Repetir,Vtuber,Cooperativo,Jogador X ambiente,Modo: Aventura,Depressão,Esports,Modo: Sobrevivência,Retrô,Jogo de guerra,Mapa: Mirage,Arte digital,Mulher,Desenho,ASMR auditivo,Para toda a família,Corrida,EUA,TDAH,Extensão aprimorada,Modo difícil,Inglaterra,Animada,Sem palpites,Espaço seguro,menor_trecho_game,maior_trecho_game,game_duration_sum_rank_1,game_duration_sum_rank_2,game_duration_sum_rank_3,game_duration_sum_rank_4,game_duration_sum_rank_5,game_duration_sum_rank_6,game_duration_sum_rank_7,game_duration_sum_rank_8,game_duration_sum_rank_9,game_duration_sum_rank_10,game_duration_count_rank_1,game_duration_count_rank_2,game_duration_count_rank_3,game_duration_count_rank_4,game_duration_count_rank_5,game_duration_count_rank_6,game_duration_count_rank_7,game_duration_count_rank_8,game_duration_count_rank_9,game_duration_count_rank_10,game_duration_mean_rank_1,game_duration_mean_rank_2,game_duration_mean_rank_3,game_duration_mean_rank_4,game_duration_mean_rank_5,game_duration_mean_rank_6,game_duration_mean_rank_7,game_duration_mean_rank_8,game_duration_mean_rank_9,game_duration_mean_rank_10,rank_jogos_recentes_1,rank_jogos_recentes_2,rank_jogos_recentes_3,rank_jogos_recentes_4,rank_jogos_recentes_5,rank_jogos_recentes_6,rank_jogos_recentes_7,rank_jogos_recentes_8,rank_jogos_recentes_9,rank_jogos_recentes_10
0,23631109,xruk,xRUK,87,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42642296,muuhpro,MuuhPro,229804,0,32,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,516587.0,516587.0,516587.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,516587.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,516587.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,516587.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
2,62916113,focasp,FocaSP,34720,0,21,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,497816.0,497816.0,497816.0,32982.0,509943.0,503924.0,2.100354e+09,494592.0,496943.0,513010.0,511922.0,376677451.0,497816.0,32982.0,503924.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.815410e+09,509943.0,2.100354e+09,32982.0,497816.0,503924.0,494592.0,496943.0,513010.0,511922.0,376677451.0,32982.0,32982.0,32982.0,32982.0,32982.0,32982.0,497816.0,2.100354e+09,497816.0,497816.0
3,29361707,zarbaofc,zarbaofc,151,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62290505,xtojah,xtojah,8694,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,

In [56]:
df_videos_all_metrics

,broadcaster_id,broadcaster_login,broadcaster_name,total_followers,team_id,qtd_emotes,Português,Brasil,Portugal,Competitivo,100%,AMA,Multijogador,LGBTQIA+,Jogada casual,Drops habilitados,Conversação,PvP,Jogando com espectadores,Anime,Jogada de ranking,Atuação e encenação,PC,Jogadores iniciantes,Primeira jogada,Futebol,Aconchegante,Relaxada,Inglês,Único jogador,Modo: Battle Royale,Repetir,Vtuber,Cooperativo,Jogador X ambiente,Modo: Aventura,Depressão,Esports,Modo: Sobrevivência,Retrô,Jogo de guerra,Mapa: Mirage,Arte digital,Mulher,Desenho,ASMR auditivo,Para toda a família,Corrida,EUA,TDAH,Extensão aprimorada,Modo difícil,Inglaterra,Animada,Sem palpites,Espaço seguro,menor_trecho_game,maior_trecho_game,game_duration_sum_rank_1,game_duration_sum_rank_2,game_duration_sum_rank_3,game_duration_sum_rank_4,game_duration_sum_rank_5,game_duration_sum_rank_6,game_duration_sum_rank_7,game_duration_sum_rank_8,game_duration_sum_rank_9,game_duration_sum_rank_10,game_duration_count_rank_1,game_duration_count_rank_2,game_duration_count_rank_3,game_duration_count_rank_4,game_duration_count_rank_5,game_duration_count_rank_6,game_duration_count_rank_7,game_duration_count_rank_8,game_duration_count_rank_9,game_duration_count_rank_10,game_duration_mean_rank_1,game_duration_mean_rank_2,game_duration_mean_rank_3,game_duration_mean_rank_4,game_duration_mean_rank_5,game_duration_mean_rank_6,game_duration_mean_rank_7,game_duration_mean_rank_8,game_duration_mean_rank_9,game_duration_mean_rank_10,rank_jogos_recentes_1,rank_jogos_recentes_2,rank_jogos_recentes_3,rank_jogos_recentes_4,rank_jogos_recentes_5,rank_jogos_recentes_6,rank_jogos_recentes_7,rank_jogos_recentes_8,rank_jogos_recentes_9,rank_jogos_recentes_10
0,23631109,xruk,xRUK,87,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42642296,muuhpro,MuuhPro,229804,0,32,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,516587.0,516587.0,516587.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,516587.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,516587.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,516587.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
2,62916113,focasp,FocaSP,34720,0,21,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,497816.0,497816.0,497816.0,32982.0,509943.0,503924.0,2.100354e+09,494592.0,496943.0,513010.0,511922.0,376677451.0,497816.0,32982.0,503924.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.815410e+09,509943.0,2.100354e+09,32982.0,497816.0,503924.0,494592.0,496943.0,513010.0,511922.0,376677451.0,32982.0,32982.0,32982.0,32982.0,32982.0,32982.0,497816.0,2.100354e+09,497816.0,497816.0
3,29361707,zarbaofc,zarbaofc,151,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62290505,xtojah,xtojah,8694,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,

In [57]:
df_videos_all_metrics.loc[df_videos_all_metrics.broadcaster_login == 'lisgood']

,broadcaster_id,broadcaster_login,broadcaster_name,total_followers,team_id,qtd_emotes,Português,Brasil,Portugal,Competitivo,100%,AMA,Multijogador,LGBTQIA+,Jogada casual,Drops habilitados,Conversação,PvP,Jogando com espectadores,Anime,Jogada de ranking,Atuação e encenação,PC,Jogadores iniciantes,Primeira jogada,Futebol,Aconchegante,Relaxada,Inglês,Único jogador,Modo: Battle Royale,Repetir,Vtuber,Cooperativo,Jogador X ambiente,Modo: Aventura,Depressão,Esports,Modo: Sobrevivência,Retrô,Jogo de guerra,Mapa: Mirage,Arte digital,Mulher,Desenho,ASMR auditivo,Para toda a família,Corrida,EUA,TDAH,Extensão aprimorada,Modo difícil,Inglaterra,Animada,Sem palpites,Espaço seguro,menor_trecho_game,maior_trecho_game,game_duration_sum_rank_1,game_duration_sum_rank_2,game_duration_sum_rank_3,game_duration_sum_rank_4,game_duration_sum_rank_5,game_duration_sum_rank_6,game_duration_sum_rank_7,game_duration_sum_rank_8,game_duration_sum_rank_9,game_duration_sum_rank_10,game_duration_count_rank_1,game_duration_count_rank_2,game_duration_count_rank_3,game_duration_count_rank_4,game_duration_count_rank_5,game_duration_count_rank_6,game_duration_count_rank_7,game_duration_count_rank_8,game_duration_count_rank_9,game_duration_count_rank_10,game_duration_mean_rank_1,game_duration_mean_rank_2,game_duration_mean_rank_3,game_duration_mean_rank_4,game_duration_mean_rank_5,game_duration_mean_rank_6,game_duration_mean_rank_7,game_duration_mean_rank_8,game_duration_mean_rank_9,game_duration_mean_rank_10,rank_jogos_recentes_1,rank_jogos_recentes_2,rank_jogos_recentes_3,rank_jogos_recentes_4,rank_jogos_recentes_5,rank_jogos_recentes_6,rank_jogos_recentes_7,rank_jogos_recentes_8,rank_jogos_recentes_9,rank_jogos_recentes_10
11798,68709676,lisgood,Lisgood,53991,0,51,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df_videos.loc[df_videos.user_id == 53052202] #tem algo de errado na base de video :'( #vou fingir qu

,id,stream_id,user_id,user_login,user_name,title,description,created_at,published_at,url,thumbnail_url,viewable,view_count,language,type,duration,muted_segments,game_id,game_name,game_duration,duration_time,duration_seconds
17213,1565919179,40090420489,53052202,boltz,boltz,DOMINGAO DO BOLTAO !keydrop !sorteio !tradeit !proskins,NaN,2022-08-18 22:59:24+00:00,2022-08-18T22:59:24Z,https://www.twitch.tv/videos/1565919179,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/2561bf4286aa7ee04de6_boltz_40090420489_1660863559//thumb/thumb0-%{width}x%{height}.jpg,public,43400.0,pt,VideoType.ARCHIVE,5h10m44s,NaN,32399.0,Counter-Strike: Global Offensive,NaN,05:10:44,18644.0
17214,1555512859,40038056777,53052202,boltz,boltz,JOGANDO RPL !keydrop !sorteio !tradeit !proskins,NaN,2022-08-08 00:50:35+00:00,2022-08-08T00:50:35Z,https://www.twitch.tv/videos/1555512859,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/2dbf09817213172dab8c_boltz_40038056777_1659919831//thumb/thumb0-%{width}x%{height}.jpg,public,42620.0,pt,VideoType.ARCHIVE,3h47m2s,NaN,32399.0,Counter-Strike: Global Offensive,NaN,03:47:02,13622.0
17215,1552593517,46930308461,53052202,boltz,boltz,JOGANDO RPL !keydrop !sorteio !tradeit !proskins,NaN,2022-08-05 00:35:53+00:00,2022-08-05T00:35:53Z,https://www.twitch.tv/videos/1552593517,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/25a521389560c242905c_boltz_46930308461_1659659748//thumb/thumb0-%{width}x%{height}.jpg,public,40488.0,pt,VideoType.ARCHIVE,4h7m56s,NaN,32399.0,Counter-Strike: Global Offensive,NaN,04:07:56,14876.0
17216,1551605877,40017648521,53052202,boltz,boltz,JOGANDO RPL !keydrop !sorteio !tradeit !proskins,NaN,2022-08-04 00:19:08+00:00,2022-08-04T00:19:08Z,https://www.twitch.tv/videos/1551605877,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/35edf03258d66f87f972_boltz_40017648521_1659572344//thumb/thumb0-%{width}x%{height}.jpg,public,40376.0,pt,VideoType.ARCHIVE,4h11m10s,NaN,32399.0,Counter-Strike: Global Offensive,NaN,04:11:10,15070.0
17217,1550821067,41069562107,53052202,boltz,boltz,JOGANDO RPL !keydrop !sorteio !tradeit !proskins,NaN,2022-08-03 04:02:09+00:00,2022-08-03T04:02:09Z,https://www.twitch.tv/videos/1550821067,https://static-cdn.jtvnw.net/cf_vods/d2nvs31859zcd8/4805d4e21fccf5e9b0b8_boltz_41069562107_1659499325//thumb/thumb0-%{width}x%{height}.jpg,public,8297.0,pt,VideoType.ARCHIVE,59m50s,NaN,32399.0,Counter-Strike: Global Offensive,NaN,00:59:50,3590.0
17218,1550627457,41068362427,53052202,boltz,boltz,DOMINGAO DO BOLTAO !keydrop !sorteio !tradeit !proskins,NaN,2022-08-03 00:18:13+00:00,2022-08-03T00:18:13Z,https://www.twitch.tv/videos/1550627457,https://static-cdn.jtvnw.net/cf_vods/d2nvs31859zcd8/38aefec6fd397fe22de0_boltz_41068362427_1659485889//thumb/thumb0-%{width}x%{height}.jpg,public,40830.0,pt,VideoType.ARCHIVE,3h42m58s,NaN,32399.0,Counter-Strike: Global Offensive,NaN,03:42:58,13378.0
17219,1548765730,40003751097,53052202,boltz,boltz,DOMINGAO DO BOLTAO !keydrop !sorteio !tradeit !proskins,NaN,2022-08-01 01:19:12+00:00,2022-08-01T01:19:12Z,https://www.twitch.tv/videos/1548765730,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/d2893fb095d9ecd751ae_boltz_40003751097_1659316748//thumb/thumb0-%{width}x%{height}.jpg,public,27681.0,pt,VideoType.ARCHIVE,2h16m18s,NaN,32399.0,Counter-Strike: Global Offensive,NaN,02:16:18,8178.0
17220,1548741504,40003590441,53052202,boltz,boltz,DOMINGAO DO BOLTAO !keydrop !sorteio !tradeit !proskins,NaN,2022-08-01 00:50:55+00:00,2022-08-01T00:50:55Z,https://www.twitch.tv/videos/1548741504,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9zdc1j/ffbb823fe32e3fd99157_boltz_40003590441_1659315050//thumb/thumb0-%{width}x%{height}.jpg,public,7953.0,pt,VideoType.ARCHIVE,27m30s,NaN,32399.0,Counter-Strike: Global Offensive,NaN,00:27:30,1650.0
17221,1545793242,41047649371,53052202,boltz,boltz,LIVE TODO DIA !keydrop !sorteio !tradeit !proskins,NaN,2022-07-29 00:16:19+00:00,2022-07-29T00:16:19Z,https://www.twitch.tv/videos/1545793242,https://static-cdn.jtvnw.net/cf_vods/d2nvs31859

In [59]:
df_videos_all_metrics_select = df_videos_all_metrics[~df_videos_all_metrics.rank_jogos_recentes_1.isna()]

In [60]:
df_videos_all_metrics_select = df_videos_all_metrics_select.fillna(0)

In [63]:
df_videos_all_metrics_select.to_csv(f'dados/all_metrics_filter.csv', mode='w', sep=";", header=True, index=False)

In [62]:
df_videos_all_metrics.to_csv(f'dados/all_metrics.csv', mode='w', sep=";", header=True, index=False)

In [ ]:
#ficou faltando extraçoes de video